## Bipartite networks from Chowdhary and Assisi, 2019

#### To-do list
3. Bipartite networks
	1. No within-group connections
	2. Complementary edges are excitatory connections
	3. Ratio of cumulative strength of excitation/inhibition is varied

#### Imports/Init

In [1]:
from brian2 import *
%matplotlib inline

INFO       Cache size for target 'cython': 1719995051 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the 'C:\Users\admin\.cython\brian_extensions' directory. [brian2]


In [ ]:
# Constants
tau = 4*ms
I = 1.1/ms
gamma = 1/ms

# Non-dimensionalized constants
tp = tau/ms
curr = I*ms
g = gamma*ms
out = curr/g

# equation (1)
eqs = '''
dv/dt = I - gamma*v : 1
'''

# equation (2) but with t replaced by (phi*tau/gamma) [para equation]
# converted to D.E. by differentiation
# dv/dt = (I/gamma) - v is the actual equation, but tau is introduced to keep the equation dimensionally consistent.
eqs2 = '''
dv/dt = ((I/gamma) - v)/tau : 1
'''

In [ ]:
# From starter.ipynb

start_scope()

exc = 0.1

G1 = NeuronGroup(1, eqs2, threshold='v>1', reset='v = 0', method='exact')
G2 = NeuronGroup(1, eqs2, threshold='v>1', reset='v = 0', method='exact')

G2.v = [0.8]

S1 = Synapses(G1, G2, on_pre='v_post = out * (1-exp(-(tp * log(1/(1 - g*(v_post + exc)/curr)))/tp))')
S2 = Synapses(G2, G1, on_pre='v_post = out * (1-exp(-(tp * log(1/(1 - g*(v_post + exc)/curr)))/tp))')

S1.connect()
S2.connect()

M1 = StateMonitor(G1, 'v', record=True)
M2 = StateMonitor(G2, 'v', record=True)
Sp1 = SpikeMonitor(G1)
Sp2 = SpikeMonitor(G2)

run(100*ms)

plot(M1.t/ms, M1.v[0], label='Neuron 1')
plot(M2.t/ms, M2.v[0], label='Neuron 2')
xlabel('Time (ms)')
ylabel('v');

In [ ]:
# ISI Plotter
spikes = min(len(Sp2.t[:]), len(Sp1.t[:]))
Vals = Sp1.t[:spikes] - Sp2.t[:spikes]

plot(Sp1.t/ms, Vals/ms)
plt.axhline(y = 0, color = 'r', linestyle = '--')
plt.axhline(y = 11/2, color = 'g', linestyle = '--')
xlabel('Time')
ylabel('Interspike Intervals')
ylim(-10, 10);